In [1]:
from ctypes import *
import numpy as np
import random

from   scipy import sparse

In [2]:
'''# speed comparison of four approaches    '''
'''    # native python libraries                        '''
'''    # specialist libraries                                   '''
'''    # ctype method calls into C                                      '''
'''    # wholesale modular call into C from compiled library                    '''
'''                                                                                 '''
'''# further optimisations maybe possible with the four approaches                      '''
'''# note that mutate request internally casts the numpy population into a list of lists    '''

'# note that mutate request internally casts the numpy population into a list of lists    '

In [3]:
class MutateRequest:
    def __init__(self, population):
        number_of_mutations = get_number_of_mutations()
        self.population     = population
        self.population     = [list(algo) for algo in population]
        self.rows_algos     = len(population)
        self.columns_genes  = len(population[0])
        self.number_of_mutations = number_of_mutations

In [4]:
class TellObject:
    def __init__(self, population, base_mutants, numpy_mutants):
        self.population     = population
        self.base_mutants   = base_mutants
        self.numpy_mutants  = numpy_mutants
        
def tell(tell_object):
    print('\noriginal population\n', np.around(np.asarray(tell_object.population), decimals=4))
    print('\nmutation by base libraries\n')
    for i in range(len(tell_object.base_mutants)):
          print(np.around(np.asarray(tell_object.base_mutants[i]),decimals=4))
    print('\nmutation by numpy/scipy libraries\n', np.around(np.asarray(tell_object.numpy_mutants), decimals=4))

In [5]:
def get_number_of_mutations():
    number_of_mutations = 3
    return number_of_mutations

In [6]:
def build_population():
    low         = -2
    high        =  2
    cohort_size = (3, 7) #  rows_algos, columns_genes
    population  = np.random.uniform(low, high, size=cohort_size)
    return population

In [7]:
def base_mutate(mutate_request):
    ''' does not use numpy; only uses naive base library - math, itertools, random etc are allowed'''
    population          = mutate_request.population
    number_of_mutations = mutate_request.number_of_mutations
    rows_algos          = mutate_request.rows_algos
    columns_genes       = mutate_request.columns_genes
    base_mutants        = population
    
    # arbitrary loop nesting order here - this is best for 
    for mutation in range(number_of_mutations):
        for algo in base_mutants:
            random_place    = random.randrange(columns_genes)
            starting_gene   = algo[random_place]
            a_float         = random.random()
            algo.pop(random_place)
            mutant_gene     = starting_gene + a_float
            algo.insert(random_place, mutant_gene)
    return base_mutants

In [8]:
def numpy_mutate(mutate_request):
    population          = mutate_request.population
    population          = np.asarray(population)
    
    number_of_mutations = mutate_request.number_of_mutations
    all_loci            = mutate_request.rows_algos * mutate_request.columns_genes
    
    mutation_rate       = number_of_mutations/mutate_request.columns_genes
    
    mutation_matrix = sparse.random(population.shape[0], population.shape[1], density=mutation_rate)
    pre_mutants     = np.copy(population)
    mutants         = np.add(pre_mutants, mutation_matrix.todense())
    return mutants

In [9]:
if __name__ == '__main__':
    
    population          = build_population()
    mutate_request      = MutateRequest(population)
    base_mutants        = base_mutate(mutate_request)
    numpy_mutants       = numpy_mutate(mutate_request)
    
    that_which_to_tell = TellObject(population, base_mutants, numpy_mutants)
    tell(that_which_to_tell)
    


original population
 [[ 0.3192  1.5408 -0.1232  0.4611 -0.7967  0.2549 -1.9794]
 [ 0.4336  1.6143  1.5183  0.659  -1.4243  1.3275  1.8823]
 [ 0.9073  1.3694  1.8177  0.6433 -0.004  -0.184   0.6032]]

mutation by base libraries

[ 0.4433  1.5408  0.3811  0.4611 -0.4015  0.2549 -1.9794]
[ 0.4336  2.6372  1.5183  0.659  -1.4243  1.6486  1.8823]
[0.9073 1.3694 2.7795 0.6433 0.7222 0.2913 0.6032]

mutation by numpy/scipy libraries
 [[ 0.8095  2.0002  0.3811  0.4611 -0.4015  0.2549 -1.9794]
 [ 0.4336  2.6372  2.2282  0.659  -1.4243  1.9774  1.8823]
 [ 0.9073  2.0198  3.1828  1.62    0.7222  1.122   0.7899]]


In [10]:
def test_base_mutate():
    number_of_mutations   = get_number_of_mutations()
    population            = build_population()
    mutate_request        = MutateRequest(population)
    mutants               = base_mutate(mutate_request)
    max_parity_sum        = population.shape[0] * population.shape[1]
    rows_algos            = mutate_request.rows_algos
    
    #form
    assert isinstance(mutants,      list),  'Error: should be a list'
    assert isinstance(mutants[0],   list),  'Error: should be a list'
    assert isinstance(mutants[0][0],float), 'Error: should be a float'
    
    #content
    flat_population = list(np.ravel(np.asarray(population)))
    flat_mutants    = list(np.ravel(np.asarray(mutants)))
    both            = zip(flat_population, flat_mutants)
    parity_check    = sum([int(x[0]==x[1]) for x in both])
    if number_of_mutations > 0:
        assert parity_check <= max_parity_sum-rows_algos, 'Error: there should be at least one difference per algo'
    assert parity_check > 0, 'Error: they should not be completely different'
    print('pass')

In [11]:
def test_numpy_mutate():
    number_of_mutations   = get_number_of_mutations()
    population            = build_population()
    mutate_request        = MutateRequest(population)
    mutants               = numpy_mutate(mutate_request)
    max_parity_sum        = population.shape[0] * population.shape[1]
    rows_algos            = mutate_request.rows_algos
    
    #form
    assert isinstance(mutants,      np.ndarray),  'Error: should be an np array'
    assert isinstance(mutants[0],   np.ndarray),  'Error: should be an np array'
    assert isinstance(mutants[0][0],np.matrix), 'Error: should be a float'
    
    #content
    flat_population = list(np.ravel(np.asarray(population)))
    flat_mutants    = list(np.ravel(np.asarray(mutants)))
    both            = zip(flat_population, flat_mutants)
    parity_check    = sum([int(x[0]==x[1]) for x in both])
    if number_of_mutations > 0:
        assert parity_check <= max_parity_sum-rows_algos, 'Error: there should be at least one difference per algo'
    assert parity_check > 0, 'Error: they should not be completely different'
    print('pass')
    

In [12]:
def test_build_population():
    population = build_population()
    assert isinstance(population,       np.ndarray),    'Error: should be array of arrays'
    assert isinstance(population[0],    np.ndarray),    'Error: should be an array'
    assert isinstance(population[0][0], np.float64),    'Error: should be a float'
    print('pass')

In [13]:
def tests():
    test_base_mutate()
    test_build_population()
    test_numpy_mutate()

In [14]:
tests()

pass
pass
pass


In [15]:
'''
# Build mutate_interface
'''

'\n# Build mutate_interface\n'

In [16]:
'''# Note

 class ctypes.Array(*args)

    Abstract base class for arrays.

    The recommended way to create concrete array types is by multiplying any ctypes data type
    with a positive integer. Alternatively, you can subclass this type and define _length_ 
    and _type_ class variables. Array elements can be read and written using standard subscript 
    and slice accesses; for slice reads, the resulting object is not itself an Array.

'''

'# Note\n\n class ctypes.Array(*args)\n\n    Abstract base class for arrays.\n\n    The recommended way to create concrete array types is by multiplying any ctypes data type\n    with a positive integer. Alternatively, you can subclass this type and define _length_ \n    and _type_ class variables. Array elements can be read and written using standard subscript \n    and slice accesses; for slice reads, the resulting object is not itself an Array.\n\n'

In [17]:
cdll.LoadLibrary("libc.so.6")

<CDLL 'libc.so.6', handle 7ff0f4115500 at 0x7ff0c85b1150>

In [18]:
libc = CDLL("libc.so.6")

In [19]:
c_wchar_p("Hello, World")

c_wchar_p(140672130292528)

In [20]:
printf = libc.printf

In [21]:
printf()

6

In [22]:
printf("Hello World")

1

In [23]:
s = "Hello, World"

In [24]:
c_s = c_wchar_p(s)

In [25]:
printf(b"Hello, %s\n", b"World!")

14

In [26]:
print(c_s.value)

Hello, World


In [27]:
print(c_s)

c_wchar_p(140672130380336)


In [28]:
print(s)                # first object is unchanged

Hello, World


In [29]:
printf(b"An int %d, a double %f\n", 1234, c_double(3.14))

31

In [30]:
strchr = libc.strchr

In [31]:
strchr(b"abcdef", ord("d"))  

-931615469

In [32]:
i = c_int()

In [33]:
f = c_float()

In [34]:
s = create_string_buffer(b'\000' * 32)

In [35]:
print(i.value, f.value, repr(s.value))

0 0.0 b''


In [36]:
libc.sscanf(b"1 3.14 Hello", b"%d %f %s", byref(i), byref(f), s)

3

In [37]:
print(i.value, f.value, repr(s.value))

1 3.140000104904175 b'Hello'


In [38]:
class POINT(Structure):
    _fields_ = [("x", c_int),
                ("y", c_int)]

In [39]:
point = POINT(10, 20)

In [40]:
print(point.x, point.y)

10 20


In [41]:
point = POINT(y=5)

In [42]:
print(point.x, point.y)

0 5


In [43]:
POINT(1, 2)

In [44]:
POINT(-1, 2)

In [45]:
#POINT(1, 2.0001)

'''
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-120-a3808173cd67> in <module>
----> 1 POINT(1, 2.0001)

TypeError: int expected instead of float


'''

'\n---------------------------------------------------------------------------\nTypeError                                 Traceback (most recent call last)\n<ipython-input-120-a3808173cd67> in <module>\n----> 1 POINT(1, 2.0001)\n\nTypeError: int expected instead of float\n\n\n'

In [46]:
# POINT(1, -2.0001)

'''
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-121-bfc065603ff4> in <module>
----> 1 POINT(1, -2.0001)

TypeError: int expected instead of float


'''

'\n---------------------------------------------------------------------------\nTypeError                                 Traceback (most recent call last)\n<ipython-input-121-bfc065603ff4> in <module>\n----> 1 POINT(1, -2.0001)\n\nTypeError: int expected instead of float\n\n\n'

In [47]:
POINT(1, 2)

In [48]:
class RECT(Structure):
    _fields_ = [("upperleft", POINT),
                ("lowerright", POINT)]

In [49]:
rc = RECT(point)

In [50]:
print(rc.upperleft.x, rc.upperleft.y)

0 5


In [51]:
print(rc.lowerright.x, rc.lowerright.y)

0 0


In [52]:
TenPointsArrayType = POINT * 10

In [53]:
r = RECT(POINT(1, 2), POINT(3, 4))

In [54]:
r = RECT((1, 2), (3, 4))

In [55]:
print(POINT.x)

<Field type=c_int, ofs=0, size=4>


In [56]:
print(POINT.y)

<Field type=c_int, ofs=4, size=4>


In [57]:
class Int(Structure):
    _fields_ = [("first_16", c_int, 16),
                ("second_16", c_int, 16)]

In [58]:
print(Int.first_16)

<Field type=c_int, ofs=0:0, bits=16>


In [59]:
print(Int.second_16)

<Field type=c_int, ofs=0:16, bits=16>


In [60]:
class POINT(Structure):
    _fields_ = ("x", c_int), ("y", c_int)

In [61]:
class MyStruct(Structure):
    _fields_ = [("a", c_int),
                ("b", c_float),
                ("point_array", POINT * 4)]

In [62]:
print(len(MyStruct().point_array))

4


In [63]:
arr = TenPointsArrayType()

In [64]:
for pt in arr:
    print(pt.x, pt.y)

0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0


In [65]:
TenIntegers = c_int * 10

In [66]:
ii = TenIntegers(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [67]:
print(ii)

In [68]:
for i in ii: print(i, end=" ")

1 2 3 4 5 6 7 8 9 10 

In [69]:
i = c_int(42)

In [70]:
pi = pointer(i)

In [71]:
pi.contents

c_int(42)

In [72]:
pi.contents is i

False

In [73]:
pi.contents is pi.contents

False

In [74]:
i = c_int(99)

In [75]:
pi.contents = i

In [76]:
pi.contents

c_int(99)

In [77]:
pi[0]

99

In [78]:
print(i)

c_int(99)


In [79]:
pi[0] = 22

In [80]:
print(i)

c_int(22)


In [81]:
PI = POINTER(c_int)

In [82]:
PI

importlib._bootstrap.LP_c_int

In [83]:
# PI(42)

'''
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-52-622c40ff30d9> in <module>
----> 1 PI(42)

TypeError: expected c_int instead of int


'''

'\n---------------------------------------------------------------------------\nTypeError                                 Traceback (most recent call last)\n<ipython-input-52-622c40ff30d9> in <module>\n----> 1 PI(42)\n\nTypeError: expected c_int instead of int\n\n\n'

In [84]:
PI(c_int(42))

In [85]:
null_ptr = POINTER(c_int)()

In [86]:
print(bool(null_ptr))

False


In [87]:
class Bar(Structure):
    _fields_ = [("count", c_int), ("values", POINTER(c_int))]

In [88]:
bar = Bar()

In [89]:
bar.values = (c_int * 3)(1, 2, 3)

In [90]:
bar.count = 3

In [91]:
for i in range(bar.count):
    print(bar.values[i])

1
2
3


In [92]:
IntArray5 = c_int * 5

In [93]:
ia = IntArray5(5, 1, 7, 33, 99)

In [94]:
qsort = libc.qsort

In [95]:
qsort.restype = None

In [96]:
CMPFUNC = CFUNCTYPE(c_int, POINTER(c_int), POINTER(c_int))



In [97]:
def py_cmp_func(a, b):
    print("py_cmp_func", a[0], b[0])
    return 0

In [98]:
cmp_func = CMPFUNC(py_cmp_func)

In [99]:
qsort(ia, len(ia), sizeof(c_int), cmp_func) 

py_cmp_func 5 1
py_cmp_func 33 99
py_cmp_func 7 33
py_cmp_func 5 7
py_cmp_func 1 7


In [100]:
def py_cmp_func(a, b):
    print("py_cmp_func", a[0], b[0])
    return a[0] - b[0]

In [101]:
qsort(ia, len(ia), sizeof(c_int), CMPFUNC(py_cmp_func)) 

py_cmp_func 5 1
py_cmp_func 33 99
py_cmp_func 7 33
py_cmp_func 1 7
py_cmp_func 5 7


In [102]:
for i in ia: print(i, end=" ")




1 5 7 33 99 

In [103]:
@CFUNCTYPE(c_int, POINTER(c_int), POINTER(c_int))
def py_cmp_func(a, b):
    print("py_cmp_func", a[0], b[0])
    return a[0] - b[0]

In [104]:
qsort(ia, len(ia), sizeof(c_int), py_cmp_func)

py_cmp_func 1 5
py_cmp_func 33 99
py_cmp_func 7 33
py_cmp_func 1 7
py_cmp_func 5 7


In [105]:
short_array = (c_short * 4)()

In [106]:
print(sizeof(short_array))

8


In [107]:
#resize(short_array, 4)

'''
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-207-5d3303a611a5> in <module>
----> 1 resize(short_array, 4)
      2 
      3 

ValueError: minimum size is 8

'''

'\n---------------------------------------------------------------------------\nValueError                                Traceback (most recent call last)\n<ipython-input-207-5d3303a611a5> in <module>\n----> 1 resize(short_array, 4)\n      2 \n      3 \n\nValueError: minimum size is 8\n\n'

In [108]:
resize(short_array, 32)

In [109]:
sizeof(short_array)

32

In [110]:
sizeof(type(short_array))

8